In [4]:
class Array(object):
    
    def __init__(self, size = 32, init = None):
        self._size = size
        self._items = [init] * size    #初始化
        
    def __getitem__(self, index):
        return self._items[index]
    
    def __setitem__(self, index, value):
        self._items[index] = value
        
    def __len__(self):
        return self._size
    
    def clear(self, value = None):
        for i in range(len(self._items)):
            self._items[i] = value
            
    def __iter__(self):
        for item in self._items:
            yield item
            
class Slot(object):
    """定义一个 hash 表 数组的槽
    注意，一个槽有三种状态，看你能否想明白。相比链接法解决冲突，二次探查法删除一个 key 的操作稍微复杂。
    1.从未使用 HashMap.UNUSED。此槽没有被使用和冲突过，查找时只要找到 UNUSED 就不用再继续探查了
    2.使用过但是 remove 了，此时是 HashMap.EMPTY，该探查点后边的元素扔可能是有key
    3.槽正在使用 Slot 节点
    """
    def __init__(self, key, value):
        self.key, self.value = key, value
        
class HashTable(object):
    
    UNUSED = None               #未被使用过
    EMPTY = Slot(None, None)    #使用过被删除
    
    def __init__(self):
        self._table = Array(8, init = HashTable.UNUSED)    #2的3次方，默认没被使用过
        self.length = 0
        
    @property
    def _load_factor(self):    #定义负载因子，超过0.8则rehashing
        return self.length / float(len(self._table))    #已使用长度/hash表总长
    
    def __len__(self):
        return self.length
    
    '''构建hash函数，根据key返回一个下标'''
    def _hash(self, key):
        return abs(hash(key)) % len(self._table)    #取内置hash函数绝对值之后对已使用长度取模
    
    '''寻找槽位置'''
    def _find_key(self, key):  
        index = self._hash(key)    #调用hash函数得到第一个槽位
        _len = len(self._table)
        while self._table[index] is not HashTable.UNUSED:    #不是未使用则往下找
            if self._table[index] is HashTable.EMPTY:        
                index = (index * 5 + 1) % _len               #Cython解决hash冲突的方式
                continue
            elif self._table[index].key == key:      #被使用过且值相等，则返回当前下标
                return index
            else:
                index = (index * 5 + 1) % _len       #已被使用则继续寻找
        return None
    
    '''确定槽是否可用'''
    def _slot_can_insert(self, index):    #确定槽是否可使用
        return (self._table[index] is HashTable.EMPTY or 
                self._table[index] is HashTable.UNUSED)
    
    '''返回可插入槽的下标'''
    def _find_slot_for_insert(self, key):
        index = self._hash(key)
        _len = len(self._table)
        while not self._slot_can_insert(index):    #当不能在该槽插入时
            index = (index * 5 + 1) % _len
        return index
    
    '''实现"in"操作符'''
    def __contains__(self, key):
        index = self._find_key(key)    #调用操作符进行寻找
        return index is not None       #返回不是None则在其中
    
    '''
    常用方法
    '''
    
    '''添加key与value'''
    def add(self, key, value):
        if key in self:    #key已在Array中
            index = self._find_key(key)
            self._table[index] = Slot(key, value)
            self.length += 1
            if self._load_factor >= 0.8:
                self._rehash()
            return True
        else:             #插入新key
            index = self._find_slot_for_insert(key)
            self._table[index] = Slot(key, value)
            self.length += 1
            if self._load_factor >= 0.8:
                self._rehash()
            return True
    
    def _rehash(self):
        old_table = self._table
        newsize = len(self._table) * 2
        self._table = Array(newsize, HashTable.UNUSED)
        
        self.length = 0
        
        for slot in old_table:
            '''对已有槽作筛选'''
            if slot is not HashTable.UNUSED and slot is not HashTable.EMPTY:
                index = self._find_slot_for_insert(slot.key)    #为key查找新下标
                self._table[index] = slot    #将旧槽插入新槽
                self.length += 1
                
    '''获取key对应的value'''            
    def get(self, key, default = None):
        index = self._find_key(key)
        if index is None:
            return default
        else:
            return self._table[index].value
        
    def remove(self, key):
        index = self._find_key(key)
        if index is None:
            raise KeyError()
        value = self._table[index].value
        self.length -= 1
        self._table[index] = HashTable.EMPTY
        return value
    
    def __iter__(self):
        for slot in self._table:
            if slot not in (HashTable.EMPTY, HashTable.UNUSED):
                yield slot.key
                
#########################################
# 上边是从 哈希表章 拷贝过来的代码，
# 我们会直接继承 HashTable 实现 dict
#########################################

class DictADT(HashTable):
    
    def _iter_slot(self):
        for slot in self._table:
            if slot not in (HashTable.EMPTY, HashTable.UNUSED):
                yield slot
                
    def __setitem__(self, key, value):
        self.add(key, value)
        
    def __getitem__(self, key):
        if key not in self:
            raise KeyError()
        else:
            return self.get(key)
        
    def items(self):
        for slot in self._iter_slot():
            yield (slot.key, slot.value)
            
    def keys(self):
        for slot in self._iter_slot():
            yield slot.key

    def values(self):
        for slot in self._iter_slot():
            yield slot.value

def test_dict_adt():
    import random
    d = DictADT()
    
    d['a'] = 1
    assert d['a'] == 1
    d.remove('a')
    
    l = list(range(30))
    random.shuffle(l)
    for i in l:
        d.add(i, i)
        
    for i in range(30):
        assert d.get(i) == i
        
        assert sorted(list(d.keys())) == sorted(l)
        
test_dict_adt()